In [92]:
START = 0
EOF = -1
KW,OP,DEL,ID,INT,FP,STR, SP_OR_COMM = 101,102,103,104,105,106,107, 108
YES_KW, YES_OP, YES_DEL, YES_ID, YES_INT, YES_FP, YES_STR, YES_SP_OR_COMM = 201, 202, 203, 204, 205, 206, 207, 208


In [ ]:
Tokens = {
    'keyword': 'Keyword',
    'op': 'Operator',
    'delim': 'Delimiter',
    'id': 'Identifier',
    'int': 'Integer Literal',
    'fp': 'Floating Point Literal',
    'str': 'String Literal'
}

charSet = [
    'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
    'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
    '0','1','2','3','4','5','6','7','8','9','_'
]

TokenNo = 1

keywords = ['int', 'float', 'boolean', 'string',
            'while', 'until', 'if else', 'true', 'false']
keywordEnm = list(enumerate(keywords, start=TokenNo))
TokenNo += len(keywords)

delimiters = ['{', '}', '[', ']', '(', ')', ';', ',']
delimEnm = list(enumerate(delimiters, start=TokenNo))
TokenNo += len(delimiters)

operators = ['+', '-', '*', '/', '%', '?', '>', '<', '!', ':',
            ':=', '==', '>=', '<=', '!=', '&&', '||']
opEnm = list(enumerate(operators, start=TokenNo))
TokenNo += len(operators)

In [ ]:
def getNext():
    return ''

In [ ]:
def util1(str, fp, state):
    return [str, fp, state]

In [111]:
def isKeyword(str):
    fp = 0
    for kw in keywords:
        if str[fp:len(kw)] is kw:
            fp+=len(kw)
            return util1(str[:fp], fp, YES_KW)
    return util1('', fp, -1)

print(isKeyword('true'))

['true', 4, 201]


In [110]:
def isDelimiter(str):
    if str[0] in delimiters:
        return util1(str[0],1,YES_DEL)
    else:
        return util1('',1,-1)

print(isDelimiter('{'))

['{', 1, 203]


In [ ]:
def isOp(str):
    state = OP
    fp = 0
    while True:
        if state == OP:
            if str[fp] in operators[:6]:
                state = 1
            elif str[fp] == ':':
                state = ':'
            elif str[fp] == '>':
                state = '>'
            elif str[fp] == '<':
                state = '<'
            elif str[fp] == '!':
                state = '!'
            elif str[fp] == '=':
                state = '=='
            elif str[fp] == '&':
                state = '&&'
            elif str[fp] == '|':
                state = '||'
            else:
                return util1('', fp, -1)
            fp+=1
            
        elif state == 1:
            return util1(str[:fp], fp, YES_OP)
        
        elif state==':' or state=='>' or state=='<' or state=='!':
            if str[fp] == '=':
                fp+=1
            return util1(str[:fp], fp, YES_OP)
        
        elif state == '==':
            if str[fp] == '=':
                fp+=1
                return util1(str[:fp], fp, YES_OP)
            else:
                return util1('', fp, -1)
            
        elif state == '&&':
            if str[fp] == '&':
                fp+=1
                return util1(str[:fp], fp, YES_OP)
            else:
                return util1('', fp, -1)
            
        elif state == '||':
            if str[fp] == '|':
                fp+=1
                return util1(str[:fp], fp, YES_OP)
            else:
                return util1('', fp, -1)
    
print(isOp('&&'))

In [109]:
def isId(str):
    state = ID
    fp = 0
    try:
        while True:
            if state == ID:
                if str[fp] in charSet[:26]:
                    state = 'final'
                    fp+=1
                else:
                    return util1('', fp, -1)
            
            elif state == 'final':
                try:
                    if str[fp] in charSet:
                        state = 'final'
                        fp+=1
                except(IndexError):
                    return util1(str[:fp], fp, YES_ID)
    
    except(IndexError):
        return util1('', EOF, -1)    

print(isId(''))

['', -1, -1]


In [103]:
def isInt(str):
    state = INT
    fp = 0
    try:
        while True:
            if state == INT:
                if str[fp]=='+' or str[fp]=='-':
                    state = 1
                    fp+=1
                elif str[fp] in charSet[53:62]:
                    state = 2
                    fp+=1
                elif str[fp] == '0':
                    state = 3
                    fp+=1
                else:
                    return util1('', fp, -1)
            
            elif state == 1:
                if str[fp] in charSet[53:62]:
                    state = 2
                    fp+=1
                else:
                    return util1('', fp, -1)
            
            elif state == 2:
                try:
                    if str[fp] in charSet[52:62]:
                        state = 2
                        fp+=1
                except(IndexError):
                    return util1(str[:fp], fp, YES_INT)
        
            elif state == 3:
                try:
                    if str[fp] == '0':
                        state = 3
                        fp+=1
                    elif str[fp] in charSet[53:62]:
                        state = 4
                        fp+=1
                except(IndexError):
                    return util1(str[:fp], fp, YES_INT)
                
            elif state == 4:
                return util1('', fp, -1)
            
    except(IndexError):
        return util1('', EOF, -1)
print(isInt('+2'))

['+2', 2, 205]


In [106]:
def isFP(str):
    state = FP
    fp = 0
    try:
        while True:
            if state == FP:
                if str[fp] == '+' or str[fp] == '-':
                    state = 1
                    fp += 1
                elif str[fp] in charSet[53:62]:
                    state = 2
                    fp += 1
                elif str[fp] == '0':
                    state = 6
                    fp += 1
                else:
                    return util1('', fp, -1)

            elif state == 1:
                if str[fp] in charSet[53:62]:
                    state = 2
                    fp += 1
                elif str[fp] == '0':
                    state = 8
                    fp+=1
                else:
                    return util1('', fp, -1)

            elif state == 2:
                if str[fp] in charSet[52:62]:
                    state = 2
                    fp += 1
                elif str[fp] == '.':
                    state = 3
                    fp+=1
                else:
                    return util1('', fp, -1)

            elif state == 3:
                if str[fp] == '0':
                    state = 4
                    fp += 1
                elif str[fp] in charSet[53:62]:
                    state = 5
                    fp+=1
                else:
                    return util1('', fp, -1)
            
            elif state == 4:
                if str[fp] in charSet[53:62]:
                    state = 5
                    fp+=1
                elif str[fp] == '0':
                    state = 4
                    fp+=1
                else:
                    return util1('', fp, -1)

            elif state == 5:
                try:
                    if str[fp] in charSet[52:62]:
                        state = 5
                        fp+=1
                except(IndexError):
                    return util1(str[:fp], fp, YES_FP)
        
            elif state == 6:
                if str[fp] == '.':
                    state = 7
                    fp+=1
                else:
                    return util1('', fp, -1)
                
            elif state == 7:
                if str[fp] in charSet[52:62]:
                    state = 5
                    fp+=1
                else:
                    return util1('', fp, -1)
                
            elif state == 8:
                if str[fp] == '.':
                    state = 3
                    fp+=1
                else:
                    return util1('', fp, -1)
    except(IndexError):
        return util1('', EOF, -1)

print(isFP('+00.2'))

['', 2, -1]


In [ ]:
def isStr(str):
    print(str)
    # str = r'"jj\nuk"'
    # print(str)
    state = STR
    fp = 0
    while True:
        if state == STR:
            if str[fp] == '"':
                state = 1
                fp+=1
            else:
                return util1('', fp, -1)
        
        elif state == 1:
            if str[fp] == '\\':
                state = 3
                fp+=1
            elif str[fp] == '"':
                state = 2
                fp+=1
            elif str[fp] not in ['\n', '\r', '\\', '"']:
                state = 1
                fp+=1
            else:
                return util1('', fp, -1)
                
        elif state == 2:
            return util1(str[:fp], fp, YES_STR)
        
        elif state == 3:
            if str[fp] in ['\\','n','r','"','t']:
                state = 1
                fp+=1
            else:
                return util1('', fp, -1)
            

with open('test2.txt', 'r') as f:
    x = f.read()
    print(x)
    print(isStr(x))


In [91]:
def isSpaceOrComm(str):
    state = SP_OR_COMM
    fp = 0
    try:
        while True:
            if state == SP_OR_COMM:
                if str[fp] == ' ':
                    state = 'sp'
                    fp+=1
                elif str[fp] == '#':
                    state = 'comm'
                    fp+=1
                else:
                    return util1('', fp, -1)
            
            elif state == 'sp':
                if str[fp] == ' ':
                    state = 'sp'
                    fp+=1
                else:
                    return util1('', fp, YES_SP_OR_COMM)
            
            elif state == 'comm':
                if str[fp] != '\n':
                    state = 'comm'
                    fp+=1
                else:
                    return util1('', fp, YES_SP_OR_COMM)
    except(IndexError):
        return
            
print(isSpaceOrComm('#sug'))

None


In [ ]:
def dfa(str):
    state = START
    results = []
    with open('test2.txt', 'r') as f:
        x = f.read()
        print(x)
        results.append(isKeyword(str))
        results.append(isOp(str))
        results.append(isDelimiter(str))
        results.append(isId(str))
        results.append(isInt(str))
        results.append(isFP(str))
        # results.append(isStr(str))

    # max = ['',-1,-1]
    # for tok in results:
    #     if tok[2]!=-1 and len(tok[0]) > len(max[0]):
    #         max = tok
    # print(max)

dfa('int{')
    